In [1]:
import string
import collections
import re
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
apple_link = '../../Datasets/SberHack/Task2/Comments/AppStore.csv'
google_link = '../../Datasets/SberHack/Task2/Comments/gp/'

In [3]:
app_store_df = pd.read_csv(apple_link)
cols = app_store_df.columns.to_list()
cols.remove('Review')
app_store_df = app_store_df.drop(cols, axis=1)
app_store_df = app_store_df.rename(columns={'Review':'Review Text'})
app_store_df

,Review Text
0,Не скачивается стала обновлять и зависло прило...
1,Приложение не открывается после обновления!!!!...
2,"Добрый день, после последнего обновления пропа..."
3,Не могу скачать так как требует подключения Wi-fi
4,Почему то не могу оплатить при минусе на балан...
5,Не могу связаться с поддержкой банка. Пишет чт...
6,"Сделайте возможность отключать эти истории, он..."
7,А я пожалуй закрою карту и все счета в сбере. ...
8,"Объясните, почему нужно оплачивать перевод ден..."
9,Дипломированные Мошенники


In [4]:
google_df = pd.DataFrame({'Review Text':[]})

for d in os.listdir(google_link):
    g_df = pd.read_csv(google_link + d, sep=',')
    cols = g_df.columns.to_list()
    cols.remove('Review Text')
    g_df = g_df.drop(cols, axis=1)
    google_df = google_df.append(g_df)

google_df

C:\Users\mdere\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Review Text
0,NaN
1,все прекрасно и четко
2,NaN
3,Супер)
4,NaN
5,NaN
6,NaN
7,все отлично
8,NaN
9,удобный интерфейс


In [5]:
google_df = google_df.dropna()
google_df

,Review Text
1,все прекрасно и четко
3,Супер)
7,все отлично
9,удобный интерфейс
11,Не хватает функций по сравнению с полной верси...
12,👍
14,последний раз пользовался этим приложением нес...
19,класс
24,"очень удобно,спасибо всем"
29,"Очень удобно, всем рекомендую."


In [6]:
all_df = app_store_df.append(google_df)
all_df

,Review Text
0,Не скачивается стала обновлять и зависло прило...
1,Приложение не открывается после обновления!!!!...
2,"Добрый день, после последнего обновления пропа..."
3,Не могу скачать так как требует подключения Wi-fi
4,Почему то не могу оплатить при минусе на балан...
5,Не могу связаться с поддержкой банка. Пишет чт...
6,"Сделайте возможность отключать эти истории, он..."
7,А я пожалуй закрою карту и все счета в сбере. ...
8,"Объясните, почему нужно оплачивать перевод ден..."
9,Дипломированные Мошенники


In [7]:
all_df['Review Text'] = all_df['Review Text'].apply(lambda x: re.sub("[^0-9a-zA-Zа-яА-Я ]", "", x))

In [8]:
all_df

,Review Text
0,Не скачивается стала обновлять и зависло прило...
1,Приложение не открывается после обновления Уже...
2,Добрый день после последнего обновления пропал...
3,Не могу скачать так как требует подключения Wifi
4,Почему то не могу оплатить при минусе на балан...
5,Не могу связаться с поддержкой банка Пишет что...
6,Сделайте возможность отключать эти истории они...
7,А я пожалуй закрою карту и все счета в сбере П...
8,Объясните почему нужно оплачивать перевод дене...
9,Дипломированные Мошенники


In [21]:
all_df['Review Text'] = all_df['Review Text'].apply(lambda x: np.nan if len(x.split(' ')) <= 5 else x)
all_df = all_df.dropna()
all_df

C:\Users\mdere\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Review Text
0,Не скачивается стала обновлять и зависло прило...
1,Приложение не открывается после обновления Уже...
2,Добрый день после последнего обновления пропал...
3,Не могу скачать так как требует подключения Wifi
4,Почему то не могу оплатить при минусе на балан...
5,Не могу связаться с поддержкой банка Пишет что...
6,Сделайте возможность отключать эти истории они...
7,А я пожалуй закрою карту и все счета в сбере П...
8,Объясните почему нужно оплачивать перевод дене...
10,Очень плохо стало работать приложение в послед...


In [23]:
all_df.to_csv('dataset.csv')

In [24]:
all_df

,Unnamed: 0,Review Text
0,0,Не скачивается стала обновлять и зависло прило...
1,1,Приложение не открывается после обновления Уже...
2,2,Добрый день после последнего обновления пропал...
3,3,Не могу скачать так как требует подключения Wifi
4,4,Почему то не могу оплатить при минусе на балан...
5,5,Не могу связаться с поддержкой банка Пишет что...
6,6,Сделайте возможность отключать эти истории они...
7,7,А я пожалуй закрою карту и все счета в сбере П...
8,8,Объясните почему нужно оплачивать перевод дене...
9,10,Очень плохо стало работать приложение в послед...
